In [14]:
from rasterio.plot import show
from rasterio.merge import merge
from rasterio.enums import Resampling
from rasterio.mask import mask
import rasterio
from pathlib import Path
import pandas as pd
import geopandas as gpd
import leafmap
import os

In [3]:
# read geojson file
collection = 'Kahramanmaras-turkey-earthquake-23'
url = leafmap.maxar_collection_url(collection, dtype='geojson')
gdf = gpd.read_file(url)
gdf.head()

,datetime,platform,gsd,ard_metadata_version,catalog_id,utm_zone,quadkey,view:off_nadir,view:azimuth,view:incidence_angle,view:sun_azimuth,view:sun_elevation,proj:epsg,grid:code,proj:bbox,tile:data_area,tile:clouds_area,tile:clouds_percent,visual,geometry
0,2021-02-28 08:10:22+00:00,WV03,0.39,0.0.1,1040010067C49900,37,120020230322,29.4,22.0,57.4,153.1,40.6,32637,MXRA-Z37-120020230322,"600452.7792440292,4199843.75,605156.25,4204327...",20.8,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...,"POLYGON ((40.19672 37.94010, 40.14382 37.94062..."
1,2021-02-28 08:10:22+00:00,WV03,0.39,0.0.1,1040010067C49900,37,120020230323,29.3,21.3,57.6,153.1,40.6,32637,MXRA-Z37-120020230323,"604843.75,4199843.75,610156.25,4204393.030737486",23.9,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...,"POLYGON ((40.25361 37.93950, 40.19317 37.94013..."
2,2021-02-28 08:10:22+00:00,WV03,0.39,0.0.1,1040010067C49900,37,120020230332,29.1,20.6,57.8,153.1,40.6,32637,MXRA-Z37-120020230332,"609843.75,4199843.75,615155.9736693815,4204458...",24.1,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...,"POLYGON ((40.25006 37.93954, 40.25075 37.98047..."
3,2021-02-28 08:10:22+00:00,WV03,0.39,0.0.1,1040010067C49900,37,120020230333,29.1,20.3,57.8,153.1,40.6,32637,MXRA-Z37-120020230333,"614843.75,4199843.75,615155.9736693815,4204458...",1.2,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...,"POLYGON ((40.30694 37.93892, 40.30768 37.98047..."
4,2021-02-28 08:10:23+00:00,WV03,0.39,0.0.1,1040010067C49900,37,120020232100,29.4,21.8,57.4,153.1,40.6,32637,MXRA-Z37-120020232100,"600503.194703785,4194843.75,605156.25,4200156.25",24.5,0.0,0,https://maxar-opendata.s3.amazonaws.com/events...,"POLYGON ((40.19677 37.94291, 40.19599 37.89504..."


In [4]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2024 entries, 0 to 2023
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   datetime              2024 non-null   datetime64[ns, UTC]
 1   platform              2024 non-null   object             
 2   gsd                   2024 non-null   float64            
 3   ard_metadata_version  2024 non-null   object             
 4   catalog_id            2024 non-null   object             
 5   utm_zone              2024 non-null   int64              
 6   quadkey               2024 non-null   object             
 7   view:off_nadir        2024 non-null   float64            
 8   view:azimuth          2024 non-null   float64            
 9   view:incidence_angle  2024 non-null   float64            
 10  view:sun_azimuth      2024 non-null   float64            
 11  view:sun_elevation    2024 non-null   float64            
 12

In [5]:
img_date_pre_df = pd.read_csv('turkey_earthquake/images/metadata/images_date_pre_disaster.csv')
img_date_post_df = pd.read_csv('turkey_earthquake/images/metadata/images_date_post_disaster.csv')
img_date_pre_df.head()

,catalog_id,date
0,10200100B5C3A800,2021-08-17
1,10300100D6740900,2022-07-20
2,10300100D6789800,2022-07-20
3,10300100D76F1300,2022-08-05
4,10300100D797E100,2022-07-26


In [6]:
def get_date_from_catalog(catalog_id, metadata_df):
    date = metadata_df[metadata_df['catalog_id']==catalog_id]['date'].values[0]
    date_str = ''.join(date.split('-'))
    return date_str

get_date_from_catalog('10300100D76F1300',img_date_pre_df)

'20220805'

In [67]:
import os
import rasterio
from rasterio.merge import merge
import numpy as np

def list_subdirectories(directory_path):
    subdirectories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]
    return subdirectories

def mosaic_crop_and_compress_tiffs(directory, output_tiff, compress='DEFLATE'):
    # Collect all TIFF files from the directory
    tiff_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.tif')]

    if not tiff_files:
        print("No TIFF files found in the directory.")
        return

    # Read the TIFFs and merge them
    tiffs_to_mosaic = [rasterio.open(tiff) for tiff in tiff_files]
    mosaic, out_trans = merge(tiffs_to_mosaic)

    # Close the individual TIFF files
    for tiff in tiffs_to_mosaic:
        tiff.close()

    # Determine the bounding box of the non-masked data
    no_data_value = 0 # Modify this if your nodata value is different
    rows, cols = np.where(mosaic[0] != no_data_value)
    min_row, max_row = rows.min(), rows.max() + 1
    min_col, max_col = cols.min(), cols.max() + 1

    # Crop the mosaic using the bounding box
    cropped_mosaic = mosaic[:, min_row:max_row, min_col:max_col]

    # Update the transform by applying the translation
#     out_trans = rasterio.transform.Affine.translation(out_trans.e + min_col * out_trans.a, out_trans.f + min_row * out_trans.e) * out_trans
    out_trans_updated = rasterio.Affine(out_trans.a, out_trans.b, out_trans.c + min_col * out_trans.a,
                                  out_trans.d, out_trans.e, out_trans.f + min_row * out_trans.e)

    # Get metadata from one of the original TIFFs
    meta = rasterio.open(tiff_files[0]).meta.copy()
    meta.update({
        "driver": "GTiff",
        "height": cropped_mosaic.shape[1],
        "width": cropped_mosaic.shape[2],
        "transform": out_trans_updated,
        "compress": compress,
        "predictor": 2
    })

    # Write the result to the output file
    with rasterio.open(output_tiff, 'w', **meta) as dest:
        dest.write(cropped_mosaic)

In [64]:
# catalog_list = list_subdirectories(input_folder)
# for catalog_id in catalog_list:
#     catalog_dir = pre_img_path+catalog_id+'/'
#     print([os.path.join(catalog_dir, file) for file in os.listdir(catalog_dir) if file.lower().endswith('.tif')])

In [65]:
city = 'Osmaniye'
path = 'turkey_earthquake/images/raw/'+city+'/'
pre_img_path = path+'pre/'
post_img_path = path+'post/'
merge_dir = 'turkey_earthquake/images/merged_data/'

In [66]:
%%time
# mosaic pre-disaster tiff
catalog_list = list_subdirectories(pre_img_path)
for catalog_id in catalog_list:
    catalog_dir = pre_img_path+catalog_id+'/'
    img_date = get_date_from_catalog(catalog_id,img_date_pre_df)
    # image name format: [City]_[ImgDate]_[Catalogid]_[pre/post]_diaster.tif
    output_path = merge_dir+city+'_'+img_date+'_'+catalog_id+'_pre_disaster'+'.tif'
    mosaic_crop_and_compress_tiffs(catalog_dir, output_path)

CPU times: user 1min 10s, sys: 19.7 s, total: 1min 29s
Wall time: 1min 43s


In [68]:
%%time
# mosaic post-disaster tiff
catalog_list = list_subdirectories(post_img_path)
for catalog_id in catalog_list:
    catalog_dir = post_img_path+catalog_id+'/'
    img_date = get_date_from_catalog(catalog_id,img_date_post_df)
    # image name format: [City]_[ImgDate]_[Catalogid]_[pre/post]_diaster.tif
    output_path = merge_dir+city+'_'+img_date+'_'+catalog_id+'_post_disaster'+'.tif'
    mosaic_crop_and_compress_tiffs(catalog_dir, output_path)

CPU times: user 2min, sys: 20.7 s, total: 2min 21s
Wall time: 2min 35s


## Begin Process HERE

In [69]:
def mosaic_images(img_dir, merge_dir, city):
    print("> Begin mosaic:",city)
    path = img_dir+city+'/'
    pre_img_path = path+'pre/'
    post_img_path = path+'post/'
    # mosaic pre-disaster tiff
    catalog_list = list_subdirectories(pre_img_path)
    for catalog_id in catalog_list:
        catalog_dir = pre_img_path+catalog_id+'/'
        img_date = get_date_from_catalog(catalog_id,img_date_pre_df)
        # image name format: [City]_[ImgDate]_[Catalogid]_[pre/post]_diaster.tif
        output_path = merge_dir+city+'_'+img_date+'_'+catalog_id+'_pre_disaster'+'.tif'
        mosaic_crop_and_compress_tiffs(catalog_dir, output_path)
    # mosaic post-disaster tiff
    catalog_list = list_subdirectories(post_img_path)
    for catalog_id in catalog_list:
        catalog_dir = post_img_path+catalog_id+'/'
        img_date = get_date_from_catalog(catalog_id,img_date_post_df)
        # image name format: [City]_[ImgDate]_[Catalogid]_[pre/post]_diaster.tif
        output_path = merge_dir+city+'_'+img_date+'_'+catalog_id+'_post_disaster'+'.tif'
        mosaic_crop_and_compress_tiffs(catalog_dir, output_path)

In [70]:
img_dir = 'turkey_earthquake/images/raw/'
city_list = list_subdirectories(img_dir)
city_list

['Osmaniye', 'Kahramanmaras']

In [ ]:
from tqdm import tqdm

for city in tqdm(city_list):
    mosaic_images(img_dir,merge_dir,city)